# Open Ai Gym cartpole 을 nature 2015  방법으로 DQN 학습!

"https://www.nature.com/articles/nature14236"

Nature 2015 keypoint:

기존에 

1. Correlations between samples
2. Non stationary target 

위와같은 이유로 Q 수렴성이 떨어지는 문제를

1. Go deep *2013
2. Capture and replay *2013
3. Separate networks *2015

의 해결방법으로 해결함
    

In [1]:
#setup

import numpy as np
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior() 

#이거 확실히 v2 랑 v1 공부해야할듯
# https://www.youtube.com/watch?v=FvAsdZwavpA
# tensorflow2.0 이 좀더 직관적이고 hihg level API 인 keras 위주로 가는것같음
# https://www.tensorflow.org/guide/migrate 이민가이드고.. 단하나 contrib 는 예욐ㅋㅋ

import random
from collections import deque




Instructions for updating:
non-resource variables are not supported in the long term


# OPEN AI 의 Cartpole 환경 도입

1. input_size = state 가 어떤걸 의미하는지 알수없는 4개의 숫자로 표현이됨. env.step(action)의 첫행임

<div class="alert alert-warning" role="alert" style="margin: 10px">

 <p>
action = env.action_space.sample()
    
observation, reward, done, _ = env.step(action)
 </p>
</div>


2. output_size = 왼쪽 혹은 오른쪽 2개
3. discount factor = 0.9


In [2]:
import gym
env =gym.make('CartPole-v0')
#env = gym.wrappers.Monitor(env, 'gym-results/', force=True)

input_size = env.observation_space.shape[0]
output_size= env.action_space.n

env._max_episode_steps = 10001

# constants difining our neural network

dis =0.9
REPLAY_MEMORY = 50000
MAX_EPISODE = 5000
BATCH_SIZE = 10
sigma = 1



# DQN class 정의

ref: https://github.com/hunkim/ReinforcementZeroToAll/blob/master/dqn.py 에서 수정

# DQN class can

1) Build network\
2) Predict Q_value given state\
3) Train parameters

Args :

 session (tf.Session): Tensorflow session\
 input_size (int): Input dimension\
 output_size (int): Number of discrete actions\
 name (str, optional): TF Graph will be built under this name scope
 
# DQN Network architecture :

Args:

 h_size (int, optional): Hidden layer dimension\
 l_rate (float, optional): Learning rate
 
acknowledgement:

hidden layer 수 = 1\
hidden layer dimension/node 수 =10\
learning rate = 0.001\
activation function = tanh\
loss function = MSE\
optimizer = adam\



In [3]:
class DQN:
    def __init__(self, session: tf.Session, input_size: int, output_size: int, name: str="main"):
        
        # tensorflow 2.0.0버전에선 tf.Session 이사용되지않음.  tf.compat.v1.Session 로 변경 했다가 위에 import 를 아예 바꿔버림
        
        self.session =session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        
        self._build_network()
        
#===============================================================================================
        
    def _build_network(self, h_size = 10, l_rate = 0.1):
    
        with tf.variable_scope(self.net_name):
            self._X = tf.placeholder( shape=[None, self.input_size], name="input_x", dtype=tf.float32)
            
            #first layer 
            W1= tf.get_variable("W1", shape=[self.input_size, h_size], 
                                initializer=tf.keras.initializers.glorot_uniform())
            
            #initializer=tf.contrib.layers.xavier_initializer()) 였지만
            # 나도 contrib 문제를 겪게되어 아영이의 initializer로 바꾸어 주었다.
            # tensorflow_core.compat.v1 조차도 contrib API가 없어져ㅆ다..
            #로할라했지만 contrib 없이 xavier Glorot (사람이름) 의 다른방법 = Glorot 이 있어서 그걸로 하였다.
            # https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform?version=stable
            
            layer1 = tf.nn.tanh(tf.matmul(self._X, W1))
            
            #Second layer             
            W2 = tf.get_variable("W2", shape=[h_size, self.output_size], 
                                 initializer=tf.keras.initializers.glorot_uniform())
            
            
            
            # Q_prediction
            self._Qpred = tf.matmul(layer1, W2)
            
            
            #we need to define the parts of the network needed for learning a policy
            self._Y = tf.placeholder(shape=[None, self.output_size], dtype=tf.float32)
            
            #loss function                   
            self._loss = tf.losses.mean_squared_error(self._Y, self._Qpred)

            #learning            
            self._train = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(self._loss)
           
            
#===============================================================================================
            
    # Q(s,a) 을 리턴 하는 predict 함수
    # state (np.ndarray): State array, shape (n, imput_dim)
    # return np.ndarray: Q value array, shape (n, output_dim)
    
    
    def predict(self, state: np.ndarray):
        
        x = np.reshape(state, [1, self.input_size])
        return self.session.run(self._Qpred, feed_dict={self._X: x})
    
#===============================================================================================
    # X, y 를 받아 업데이트 하고 loss, result from train step 을 리턴
    
    
    def update(self, x_stack: np.ndarray, y_stack: np.ndarray):
      
        return self.session.run([self._loss, self._train], feed_dict={self._X: x_stack, self._Y: y_stack})

# Train from replay buffer (Simple Replay Train)

In [4]:
def simple_replay_train(DQN,train_batch):
    x_stack = np.empty(0).reshape(0, DQN.input_size)
    y_stack = np.empty(0).reshape(0, DQN.output_size)
    
    #get stored information from the buffer
    for state, action, reward, next_state, done in train_batch:
        Q=DQN.predict(state)
        
        #terminal?
        if done:
            Q[0,action] = reward
        else:
            #obtain Q' values by feeding the new state through our network
            Q[0,action] =reward + dis*np.max(DQN.predict(next_state))
            
        x_stack = np.vstack([x_stack, state])    
        y_stack = np.vstack([y_stack, Q])
        
    #train our network using target and predicted Q values on each episode
    return DQN.update(x_stack, y_stack)

# 학습된 네트워크를 받아 play 를 하는 함수

In [5]:
def bot_play(mainDQN):
    s=env.reset()
    reward_sum = 0
    while True:
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s, reward, done, _ =env.step(a)
        reward_sum += reward
        if done:
            print("Total score: {}".format(reward_sum))
            env.close()  #안하면 응답없음 걸려버림
            break
    

# Main 코드

In [6]:
def main():
    
    
    # store the previous observations in replay memory
    
    replay_buffer = deque(maxlen=REPLAY_MEMORY)
    last_100_game_reward = deque(maxlen=100)

    with tf.Session() as sess:
        mainDQN = DQN(sess, input_size, output_size)
        
        tf.global_variables_initializer().run()    # Weight 초기화
        
       

        for episode in range(MAX_EPISODE):
            e = 1. / ((episode/ 10)+1) #decaying epsilon
            done = False
            state = env.reset()
            step_count = 0
            
                    
            while not done:
                
                #epillon 의 확률로 random action 을하고 나머지 확률로 argmax
                if np.random.rand() < e:
                    action = env.action_space.sample() 
                else:
                    action = np.argmax(mainDQN.predict(state))
                    
                #get new state, reward from environment
                next_state, reward, done, _ = env.step(action)

                #끝나지않는게 목적이므로 패널티
                if done:
                    reward = -100

                    
                #경험을 replay_buffer 에 저장. 위의 deque 에서 replay memory 크기를 50000으로 한정
                replay_buffer.append((state, action, reward, next_state, done))
                
                state = next_state
                step_count += 1
                
                       
                
            if episode % 10 ==1:
                for _ in range(50):
                    minibatch = random.sample(replay_buffer, BATCH_SIZE)
                    loss, _ =simple_replay_train(mainDQN, minibatch)
                
                    
            if episode % 5 ==1:
                print("[Episode {:>5}]  steps: {:>5} Loss: {:4f} ".format(episode, step_count, loss))
                #print("Loss: ", loss)
            
            last_100_game_reward.append(step_count)
            #print(last_100_game_reward)

            if len(last_100_game_reward) == last_100_game_reward.maxlen:
                avg_reward = np.mean(last_100_game_reward)

                if avg_reward >3000:
                    print(f"Game Cleared in {episode} episodes with avg reward {avg_reward}")
                    break
          
        
        bot_play(mainDQN)
        

                    

In [7]:
main()

[Episode     1]  steps:    22 Loss: 633.896790 
[Episode     6]  steps:     9 Loss: 633.896790 
[Episode    11]  steps:    11 Loss: 353.407898 
[Episode    16]  steps:    58 Loss: 353.407898 
[Episode    21]  steps:    67 Loss: 6.786071 
[Episode    26]  steps:   101 Loss: 6.786071 
[Episode    31]  steps:   155 Loss: 2.982317 
[Episode    36]  steps:    82 Loss: 2.982317 
[Episode    41]  steps:    64 Loss: 576.675659 
[Episode    46]  steps:   110 Loss: 576.675659 
[Episode    51]  steps:   113 Loss: 2.328657 
[Episode    56]  steps:    45 Loss: 2.328657 
[Episode    61]  steps:    44 Loss: 1.510682 
[Episode    66]  steps:    58 Loss: 1.510682 
[Episode    71]  steps:    13 Loss: 1.428538 
[Episode    76]  steps:    57 Loss: 1.428538 
[Episode    81]  steps:    57 Loss: 0.911943 
[Episode    86]  steps:     9 Loss: 0.911943 
[Episode    91]  steps:     9 Loss: 544.165833 
[Episode    96]  steps:    84 Loss: 544.165833 
[Episode   101]  steps:    54 Loss: 2.439790 
[Episode   106]  s

# 마치며..
1. 매번 할때마다 잘학습할때도 있고 10근처에서 계속 놀아날때도 있다
   initializer 로 xavier 안쓰니까 그런거같다.\
   - 라고 생각했지만 glorot 방식으로 다시 코딩해도 비슷한결과다. 현상은 초기 initialization 이 문젠거같은데..
   - 라고 생각했지만 역시 2013년 버전이라서 non-stationary target 으로 인한 Q 가 Diverge 하는 이유가 큰거같다.
   - 뭔가 수렴하는데 필요한 episode 가 굉장히 천차만별인 느낌.
   - main 함수에 미니배치 한번에 50번학습시키는 for _ in range(50): 만들었더니 더 잘학습한다.
2. rendering 이 너무 빨리끝나버려서 잘 못본다. 마지막 결과를 반복적으로 보는 기능을 찾고싶다.
3. tensorflow 2.0 이 1.0을 포함하기도하지만 완전 리메이크버전인느낌도 든다.. 공부해야할듯